In [ ]:
# N-Steps Lookahead Agent Implemented With Numpy, Alpha/Beta Pruning, and Optimizations

In [ ]:
# imports
import numpy as np # importing numpy
from kaggle_environments import make, evaluate # importing game environment
import inspect # submission file

In [ ]:
def my_agent(obs, config) -> int:
    import numpy as np # just in case
    
    # utilites and helpers
    
    def drop_piece(board: np.ndarray, column: int, mark: int) -> np.ndarray:
        if board[0,column] != 0: #is_illegal_action = board[0, column]...
            raise ValueError("Illegal Action!")
        next_board = board.copy()
        row = np.max(np.where(board[:, column] == 0))#[0][-1]
        next_board[row, column] = mark
        return next_board
    
    def count_marks(board: np.ndarray, num_marks: int, mark: int, win_cond: int = 4) -> int:
        #function to check marks and empty positions
        count = 0
        mark_check = lambda window: np.sum(window == mark) == num_marks
        space_check = lambda window: np.sum(window == 0) == win_cond - num_marks

        # row and column check
        for direction in [board, board.T]:
            for i in range(direction.shape[0] - win_cond + 1):
                for j in range(direction.shape[1] - win_cond + 1):
                    window = direction[i:i+win_cond, j:j+win_cond]
                    count += mark_check(window) and space_check(window)
                    
        # diagonal check
        for i in range(board.shape[0] - win_cond + 1):
            for j in range(board.shape[1] - win_cond + 1):
                window = board[i:i+win_cond, j:j+win_cond]
                ascending_diag = window.diagonal()
                descending_diag = np.fliplr(window).diagonal()
                count += (mark_check(ascending_diag) and space_check(ascending_diag)) or \
                         (mark_check(descending_diag) and space_check(descending_diag))
        return count
    
    # function to check for 4 marks in a row. If found returns True
    def is_winner(board: np.ndarray, mark: int, win_cond: int = 4) -> bool:
        target = np.array([mark] * win_cond) # winning combo
        
        # row check
        for i in range(board.shape[1] - win_cond + 1):
            found = np.any(np.all(target == board[:, i:i+win_cond], axis = 1))
            if found:
                return True
        # column check
        for j in range(board.shape[0] - win_cond + 1):
            found = np.any(np.all(target == board[j:j+win_cond, :].T, axis = 1))
            if found:
                return True
        for i in range(board.shape[0] - win_cond + 1):
            for j in range(board.shape[1] - win_cond + 1):
                ascending_diag = board[i:i+win_cond, j:j+win_cond].diagonal()
                descending_diag = np.fliplr(board[i:i+win_cond, j:j+win_cond]).diagonal()
                if np.all(ascending_diag == target) or np.all(descending_diag == target):
                    return True
        return False # returns false if no rows of 4 are found
    
    #definition to determine if game is over
    def game_finished(board: np.ndarray, win_cond: int = 4) -> bool:
        return (board == 0).sum() == 0 or \
            is_winner(board, 1, win_cond) or \
            is_winner(board, 2, win_cond)
    
    # scores the detected patterns on the board
    def score_board(board: np.array) -> float:
        
        pattern_scores = {
            4: 1e10, # 4 of your marks in a line
            3: 1e5, # 3 of your marks in a line
            2: 1e2,# 2 of your marks in a line
            -2: -1, # 2 of opponents marks in a line
            -3: 1e6,# 3 of opponents marks in a line
            -4: 1e8 # 4 of opponents marks in a line
        }
        
        weighted_score = 0
        for pattern, pattern_score in pattern_scores.items():
            mark = 1 if pattern > 0 else - 1
            counts = count_marks(board, abs(pattern), mark)
            weighted_score += counts * pattern_score
        return weighted_score
    
    # function used to determine if the opponent is about to win
    def is_critical(board: np.ndarray, mark: int, win_cond: int = 4) -> bool:
        for c in range(board.shape[1]):
            for r in range(board.shape[0] - 1, -1, -1):  # Starting from bottom row

                # horizontal win check
                if c + win_cond <= board.shape[1]:
                    window = board[r, c:c+win_cond]
                    if (window == mark).sum() == win_cond - 1 and (window == 0).sum() == 1:
                        if r == board.shape[0] - 1 or board[r+1, c + np.where(window == 0)[0][0]] != 0:
                            return True

                # vertical win check
                if r >= win_cond - 1:
                    window = board[r - win_cond + 1: r + 1, c]
                    if (window == mark).sum() == win_cond - 1 and (window == 0).sum() == 1:
                        return True

                # diagonal win check (down-right)
                if r + win_cond <= board.shape[0] and c + win_cond <= board.shape[1]:
                    window = board[range(r, r + win_cond), range(c, c + win_cond)]
                    if (window == mark).sum() == win_cond - 1 and (window == 0).sum() == 1:
                        empty_index = np.where(window == 0)[0][0]
                        if r + empty_index == board.shape[0] - 1 or board[r + empty_index + 1, c + empty_index] != 0:
                            return True

                # diagonal win check (up-right)
                if r - win_cond + 1 >= 0 and c + win_cond <= board.shape[1]:
                    window = board[range(r, r - win_cond, -1), range(c, c + win_cond)]
                    if (window == mark).sum() == win_cond - 1 and (window == 0).sum() == 1:
                        empty_index = np.where(window == 0)[0][0]
                        if r - empty_index == 0 or board[r - empty_index - 1, c + empty_index] != 0:
                            return True

        return False


    #implementation of minmax search with alpha beta pruning
    def minmax_search(board: np.ndarray, depth: int, alpha: float, beta: float, is_maxing: bool) -> float:
        if depth == 0 or game_finished(board):
            return score_board(board)
        #print(is_critical(board, -1))
        if is_critical(board, -1): return -np.Inf #if opponent is about to win reutn -inf
        legal_actions = np.where(board[0] == 0)[0]
        if is_maxing:
            value = - np.Inf
            for action in legal_actions:
                child_board = drop_piece(board, action, 1)
                value = max(value, minmax_search(child_board, depth-1, alpha, beta, False))
                alpha = max(alpha, value)
                if alpha >= beta: break # beta cut-off
            return value
        else :
            value = np.Inf
            for action in legal_actions:
                child_board = drop_piece(board, action, -1) 
                value = min(value, minmax_search(child_board, depth-1, alpha, beta, True))
                beta = min(beta, value)
                if beta <= alpha: break #alpha cut-off
            return value
        
    # function to go n steps down game tree, score leaf nodes, then go up tree using minmax search
    # returns all available scores in current turn.
    def compute_scores(board: np.ndarray, n: int) -> np.array:
        scores = np.full(board.shape[1], -np.Inf)
        legal_actions = np.where(board[0] == 0)[0]
        for action in legal_actions:
            next_board = drop_piece(board, action, 1)
            scores[action] = minmax_search(next_board, n-1, -np.Inf, np.Inf, False)
        return scores
    
    # N-steps lookahead agent 
    try :
        use_board = np.array(obs.board).reshape(config.rows, config.columns)
        # board encoding, -1 = opponent mark, 1 = agent mark
        opponent = 3 - obs.mark
        use_board[use_board == opponent] = -1
        use_board[use_board == obs.mark] = 1
        
        # agent selecting action
        depth = 2
        scores = compute_scores(use_board, depth)
        legal_actions = np.where(use_board[0] == 0)[0]
        legal_scores = scores[legal_actions]
        
        if np.any(legal_scores > -np.Inf): # choose best legal action that does not lose
            best_actions = legal_actions[np.where(legal_scores == np.amax(legal_scores))[0]]
            best_ovr_action = best_actions[np.argmin(abs(best_actions - config.columns//2))]
        else: # if all legal moves result in a loss, choose random legal move 
             best_ovr_action = np.random.choice(legal_actions)
        
        #best_ovr_action = np.random.choice(best_scored_actions)
        return int(best_ovr_action)
    
    # just in case
    except Exception as e: # exception to prevent the agent from making illegal moves
        print(e)
        legal_actions = np.where(use_board[0] == 0)[0]
        #legal_ovr_action = legal_actions[np.argmin(abs(legal_actions - config.columns//2))]
        return int(np.random.choice(legal_actions))
        #return int(legal_ovr_action)
    

In [ ]:
# make game environment
# set debug = True to see errors if agent refuses to run
env = make('connectx', debug = True)

# play : agent vs random
env.run([my_agent, 'negamax'])

# render game
env.render(mode = 'ipython')

In [ ]:
# custom function used to calculate win percentage of model 
# and to determine invalid moves.
def get_win_percentages(agent1, agent2, n_rounds=100):
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print()
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(my_agent, 'negamax', n_rounds = 25)